# We first need all the datasets related to the province in which we are working like in Islamabad for Now

In [ ]:
import osmnx as ox
import folium
import pandas as pd


In [ ]:

# Define Islamabad
# place_name = "Islamabad, Pakistan"

# # Get Islamabad boundary
# gdf = ox.geocode_to_gdf(place_name)
# center = gdf.geometry.centroid.iloc[0]
# lat, lon = center.y, center.x

# # Create folium map
# m = folium.Map(location=[lat, lon], zoom_start=13, tiles='OpenStreetMap')

# # -------------------------------
# # 1. Points of Interest (blue)
# # -------------------------------
# # Get all POIs like shops, offices, schools, etc.
# tags_poi = {"amenity": True, "shop": True, "tourism": True}
# pois = ox.features_from_place(place_name, tags_poi)

# for _, row in pois.iterrows():
#     if row.geometry.geom_type == 'Point':
#         folium.CircleMarker(
#             location=[row.geometry.y, row.geometry.x],
#             radius=3,
#             color='blue',
#             fill=True,
#             fill_opacity=0.6,
#             popup=row.get('name', 'POI')
#         ).add_to(m)

# # -------------------------------
# # 2. Road Network (green lines)
# # -------------------------------
# G = ox.graph_from_place(place_name, network_type='drive')
# edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# for _, row in edges.iterrows():
#     coords = list(row.geometry.coords)
#     folium.PolyLine(locations=[(lat, lon) for lon, lat in coords], color='green', weight=1.5).add_to(m)

# # -------------------------------
# # 3. Non-constructive areas (red)
# # -------------------------------
# # These include parks, lakes, etc.
# tags_nca = {
#     "leisure": ["park", "nature_reserve", "garden"],
#     "landuse": ["forest", "recreation_ground", "grass", "meadow"],
#     "natural": ["water", "wetland"]
# }
# nca = ox.features_from_place(place_name, tags_nca)

# for _, row in nca.iterrows():
#     if row.geometry.geom_type in ['Polygon', 'MultiPolygon']:
#         folium.GeoJson(
#             row.geometry,
#             style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 1, 'fillOpacity': 0.5}
#         ).add_to(m)

# # -------------------------------
# # Save or show
# # -------------------------------
# m.save("islamabad_visualized.html")

In [ ]:
#m

In [ ]:
place = "Islamabad, Pakistan"
tags = {
    'amenity': ['restaurant', 'fast_food', 'school', 'bank', 'hospital', 'cafe'],
    'shop': True
}

# Make sure osmnx version is 1.1.0 or later
pois = ox.features_from_place(place, tags)  # ✅ correct method in latest version
pois[['geometry', 'name', 'amenity', 'shop']].to_csv('pois.csv')

In [ ]:
df = pd.read_csv("pois.csv")

In [ ]:
df.head(10)

In [ ]:
df.sample(2)

In [ ]:
import pandas as pd
from shapely import wkt
from shapely.geometry import Point, Polygon

# Load your raw pois.csv
df = pd.read_csv("pois.csv")

# Drop rows where both amenity and shop are missing
df = df.dropna(subset=["amenity", "shop"], how='all')

# Merge amenity and shop into one clean 'category' column
df["category"] = df["amenity"].fillna(df["shop"])

# Drop original columns (optional)
df = df.drop(columns=["amenity", "shop"])

# Convert geometry WKT to shapely objects
df["geometry"] = df["geometry"].apply(wkt.loads)

# Extract lat/lon from geometry
def get_lat(geom):
    if isinstance(geom, Point):
        return geom.y
    elif isinstance(geom, Polygon):
        return geom.centroid.y
    return None

def get_lon(geom):
    if isinstance(geom, Point):
        return geom.x
    elif isinstance(geom, Polygon):
        return geom.centroid.x
    return None

df["lat"] = df["geometry"].apply(get_lat)
df["lon"] = df["geometry"].apply(get_lon)

# Drop geometry if you don't need full shape anymore
df = df.drop(columns=["geometry"])

# Optional: rename or reorder columns
df = df[["element", "id", "name", "category", "lat", "lon"]]

# Save cleaned POIs
df.to_csv("pois_cleaned.csv", index=False)

print("✅ Cleaned POIs (points + polygons) saved as pois_cleaned.csv")


In [ ]:
# Define the area — you can change this to "Taxila, Pakistan" or a bounding box
place_name = "Islamabad, Pakistan"

# Extract drivable road network graph
G = ox.graph_from_place(place_name, network_type='drive')

# Convert to GeoDataFrame (only edges — roads)
edges = ox.graph_to_gdfs(G, nodes=False)

# Optional: Select only useful columns to save
columns_to_save = ['length', 'highway', 'geometry']
edges = edges[columns_to_save]

# Save to CSV
edges.to_csv("road_network_edges.csv", index=False)

print("✅ Road network saved to road_network_edges.csv")


In [ ]:
import osmnx.features as oxf  # ✅ required for feature extraction

tags = {'landuse': True}

# Extract land use geometries
landuse = oxf.features_from_place(place_name, tags)

# Optional: Select relevant columns
landuse_filtered = landuse[['landuse', 'geometry']].copy()

# Save to CSV
landuse_filtered.to_csv("landuse.csv", index=False)

print("✅ Land use data saved to landuse.csv")

In [ ]:
new_tags = {'building': True}

buildings = oxf.features_from_place(place_name, new_tags)

fil = buildings[['building', 'geometry']].copy()
fil.to_csv('buildings_data.csv', index = False)


In [ ]:
import shutil

shutil.rmtree("C:/Users/tehma/OneDrive/Desktop/Shop Prediction Model/SHOP_PROJECT/cache")

In [ ]:
df_new = pd.read_csv("pois_cleaned.csv")
df_new.head()

In [ ]:
df_new.sample(10)

In [ ]:
df_new.describe()